# Computing daily ONN index on Binance

## Getting all available symbols on Binance

In this section, we just get all available assets on Binance, generating at the end a file named `all-binance-tokens.csv`. Each line in this file contains the asset symbol two times, separated by comma (e.g. `ETH,ETH`). This is because we will use it afterwards in the __[`bnbFetchCoinData.py`](https://github.com/srmq/bnbfetch-coin-data)__ script, which expects the CSV file of lines in the format `asset_description,asset_symbol`, and we don't care about descriptions here.

In [18]:
import httpx
import pandas as pd

In [2]:
binanceBaseURL = 'https://api.binance.com'

In [3]:
async def getExchangeInfo():
    spotExchangeInfoURL = binanceBaseURL + "/api/v3/exchangeInfo"

    async with httpx.AsyncClient() as client:
        r = await client.get(spotExchangeInfoURL)
    return r.json()

In [4]:
exchangeInfo = await getExchangeInfo()

In [5]:
#blacklist = None
#with open('blacklist.txt', 'r') as f:
#    blacklist = f.read().split(',')
# blacklist should be disabled for now, we need coin data for every coin

In [6]:
baseAssets = set()
#for coinSymbol in exchangeInfo['symbols']:
#    if not (coinSymbol['baseAsset'].lower() in blacklist):
#        baseAssets.add(coinSymbol['baseAsset'])

In [7]:
for coinSymbol in exchangeInfo['symbols']:
    baseAssets.add(coinSymbol['baseAsset'])

In [8]:
def notLeveraged(asset):
    return not (asset.endswith('UP') or asset.endswith('DOWN') or asset.endswith('BEAR') or asset.endswith('BULL'))

In [9]:
baseAssets = set(filter(notLeveraged, baseAssets))

In [10]:
baseAssets

{'1INCH',
 'AAVE',
 'ACA',
 'ACH',
 'ACM',
 'ADA',
 'ADX',
 'AE',
 'AERGO',
 'AGI',
 'AGIX',
 'AGLD',
 'AION',
 'AKRO',
 'ALCX',
 'ALGO',
 'ALICE',
 'ALPACA',
 'ALPHA',
 'ALPINE',
 'AMB',
 'AMP',
 'ANC',
 'ANKR',
 'ANT',
 'ANY',
 'APE',
 'API3',
 'APPC',
 'AR',
 'ARDR',
 'ARK',
 'ARN',
 'ARPA',
 'ASR',
 'AST',
 'ASTR',
 'ATA',
 'ATM',
 'ATOM',
 'AUCTION',
 'AUD',
 'AUDIO',
 'AUTO',
 'AVA',
 'AVAX',
 'AXS',
 'BADGER',
 'BAKE',
 'BAL',
 'BAND',
 'BAR',
 'BAT',
 'BCC',
 'BCD',
 'BCH',
 'BCHA',
 'BCHABC',
 'BCHSV',
 'BCN',
 'BCPT',
 'BDOT',
 'BEAM',
 'BEL',
 'BETA',
 'BETH',
 'BGBP',
 'BICO',
 'BIFI',
 'BKRW',
 'BLZ',
 'BNB',
 'BNT',
 'BNX',
 'BOND',
 'BOT',
 'BQX',
 'BRD',
 'BSW',
 'BTC',
 'BTCB',
 'BTCST',
 'BTG',
 'BTS',
 'BTT',
 'BTTC',
 'BURGER',
 'BUSD',
 'BZRX',
 'C98',
 'CAKE',
 'CDT',
 'CELO',
 'CELR',
 'CFX',
 'CHAT',
 'CHESS',
 'CHR',
 'CHZ',
 'CITY',
 'CKB',
 'CLOAK',
 'CLV',
 'CMT',
 'CND',
 'COCOS',
 'COMP',
 'COS',
 'COTI',
 'COVER',
 'CREAM',
 'CRV',
 'CTK',
 'CTSI',
 'CTXC

In [11]:
with open('all-binance-tokens.csv', 'w') as f:
    for asset in baseAssets:
        f.write(f"{asset},{asset}\n")

Now that we have our `all-binance-tokens.csv` file, we can download, for instance, the complete historical data for all assets on Binance, using something like:

`python <path_to_script>/bnbFetchCoinData.py --coins all-binance-tokens.csv --interval 1d`

This will create a `coindata` directory under the current one with everything we need.

## Importing Data to QuestDB

We will use __[QuestDB](https://questdb.io/)__ in order to facilitate data manipulation and keep memory requirements low (we won't need to load everything on RAM). First we will create a table for holding the data. Here is what we did:

`CREATE TABLE binance_daily(symb SYMBOL capacity 2048, open_time TIMESTAMP, high DOUBLE, low DOUBLE, close DOUBLE, volume DOUBLE, close_time TIMESTAMP, quote_asset_vol DOUBLE, num_trades LONG, takerbuy_baseasset_vol DOUBLE, takerbuy_quoteasset_vol DOUBLE), index(symb) timestamp(open_time) PARTITION BY YEAR;`

Now we will import the data (that went to the `coindata` subdirectory)...

In [1]:
import glob

allCSVFiles = glob.glob('./coindata/*.csv')

In [2]:
import httpx
import csv

host = 'http://localhost:9000'

async def questInsert(symbol: str, filename: str, lotSize=300):
    async def executeQuery(valStrings):
        sql_query = "INSERT INTO binance_daily VALUES"
        sql_query += valStrings[0]
        for i in range(1, len(valStrings)):
            sql_query += f",{valStrings[i]}"
        query_params = {'query': sql_query, 'fmt': 'json'}
        async with httpx.AsyncClient() as client:
            await client.get(host + '/exec', params=query_params)
        
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        valStrings = []
        for row in reader:
            if len(valStrings) == lotSize:
                await executeQuery(valStrings)
                valStrings = []
            val = f"(\'{symbol}\', {int(row['Open time'])*1000}, {float(row['High'])}, {float(row['Low'])}, {float(row['Close'])}, {float(row['Volume'])}, {int(row['Close time'] + '999')}, {float(row['Quote asset volume'])}, {int(row['Number of trades'])}, {float(row['Taker buy base asset volume'])}, {float(row['Taker buy quote asset volume'])})"
            valStrings.append(val)
        if len(valStrings) > 0:
            await executeQuery(valStrings)

In [3]:
from pathlib import Path
i = 0
for csvFile in allCSVFiles:
    symb = Path(csvFile).name.split('.')[0]
    await questInsert(symb, csvFile)
    i += 1
    if i % 100 == 0:
        print(f"Inserted {i} symbols out of {len(allCSVFiles)}")    

Inserted 100 symbols out of 1819
Inserted 200 symbols out of 1819
Inserted 300 symbols out of 1819
Inserted 400 symbols out of 1819
Inserted 500 symbols out of 1819
Inserted 600 symbols out of 1819
Inserted 700 symbols out of 1819
Inserted 800 symbols out of 1819
Inserted 900 symbols out of 1819
Inserted 1000 symbols out of 1819
Inserted 1100 symbols out of 1819
Inserted 1200 symbols out of 1819
Inserted 1300 symbols out of 1819
Inserted 1400 symbols out of 1819
Inserted 1500 symbols out of 1819
Inserted 1600 symbols out of 1819
Inserted 1700 symbols out of 1819
Inserted 1800 symbols out of 1819


Now QuestDB has ingested all data. Let's play!

## Playing with the data

In [85]:
import pandas as pd
import httpx

In [86]:
host = 'http://localhost:9000'

In [87]:
async def daysTradedInMonthBySymbol(year: int, month: int) -> pd.DataFrame:
    assert(month >= 1 and month <= 12)
    assert(year >= 2000 and year <= 9999)
    if month == 12:
        nextMonth = 1
        nextMonthYear = year + 1
    else:
        nextMonth = month + 1
        nextMonthYear = year
    sql_query = "select symb, count(*) as daystraded from 'binance_daily' where open_time >= '%d-%02d-01T00:00:00.000000Z' and open_time < '%d-%02d-01T00:00:00.000000Z' and num_trades >= 1 group by symb"%(year, month, nextMonthYear, nextMonth)
    query_params = {'query': sql_query, 'fmt': 'json'}
    async with httpx.AsyncClient() as client:
            r = await client.get(host + '/exec', params=query_params)
    jsonR = r.json()
    return pd.DataFrame(columns=['symb', 'daysTraded'], data=jsonR['dataset'])

In [88]:
# simple test for daysTradedInMonthBySymbol
resp = await daysTradedInMonthBySymbol(2020, 11)
resp

,symb,daysTraded
0,FTTBNB,30
1,WBTCBTC,30
2,WINUSDC,30
3,BELBTC,30
4,TRBUSDT,30
...,...,...
812,AVAUSDT,7
813,HEGICETH,6
814,HEGICBUSD,6
815,PROMBUSD,5


In [89]:
import calendar
import datetime
from typing import List

# we will consider that an eligible asset should have at least one trade
# pair that was negotiated every day of the month
async def symbolsTradedEveryDay(year: int, month: int) -> List[str]:
    firstDayOfMonth = datetime.date(year, month, 1)
    daysInMonth = calendar.monthrange(firstDayOfMonth.year, firstDayOfMonth.month)[1]
    df = await daysTradedInMonthBySymbol(year, month)
    return df[df['daysTraded']==daysInMonth]['symb'].tolist()

In [90]:
# simple test for symbolsTradedEveryDay
await symbolsTradedEveryDay(2020, 11)

['FTTBNB',
 'WBTCBTC',
 'WINUSDC',
 'BELBTC',
 'TRBUSDT',
 'BTCGBP',
 'BCHBNB',
 'WTCBTC',
 'BCHUSDC',
 'QKCETH',
 'SCUSDT',
 'XRPBNB',
 'NKNUSDT',
 'WAVESETH',
 'AUDIOBUSD',
 'SOLBNB',
 'BUSDUSDT',
 'NPXSUSDT',
 'BEAMUSDT',
 'YFIIBUSD',
 'BLZBTC',
 'BNBBKRW',
 'EOSUSDT',
 'BNBPAX',
 'EOSBTC',
 'ZENBTC',
 'NANOBUSD',
 'OXTBTC',
 'BELBUSD',
 'ADAETH',
 'BCPTBTC',
 'DIABTC',
 'USDTBKRW',
 'LSKETH',
 'BTTBUSD',
 'NKNBNB',
 'LTCUSDT',
 'WAVESUSDT',
 'LOOMBTC',
 'DATABUSD',
 'CDTBTC',
 'WTCBNB',
 'RVNUSDT',
 'PNTUSDT',
 'ORNBTC',
 'THETAUSDT',
 'SRMBTC',
 'BTCUSDT',
 'STORJBTC',
 'ORNUSDT',
 'AUDIOBTC',
 'SWRVBNB',
 'RLCBTC',
 'LRCETH',
 'SANDBUSD',
 'SUSHIBUSD',
 'WRXUSDT',
 'CMTBTC',
 'PERLBTC',
 'XRPAUD',
 'WNXMBTC',
 'SNXUSDT',
 'FETUSDT',
 'STPTUSDT',
 'YFIBUSD',
 'NBSUSDT',
 'TRBBTC',
 'NEOUSDC',
 'MKRBUSD',
 'NKNBTC',
 'GOBTC',
 'ENJETH',
 'APPCBTC',
 'OCEANUSDT',
 'MDTBNB',
 'UMABTC',
 'POWRBTC',
 'XVGBTC',
 'ARPABTC',
 'VIDTBUSD',
 'ADABNB',
 'ANTBNB',
 'MFTETH',
 'DENTUSDT',
 'ZRX

In [35]:
import json
with open('coindata/coin-metadata.json', 'r') as f:
    coinMetadata = json.loads(f.read())
symbols = coinMetadata['exchange-info']['symbols']
symbolDict = {}
symbolsByBaseDict = {}
for symbol in symbols:
    symbolDict[symbol['symbol']] = symbol
    baseSymbols = symbolsByBaseDict.get(symbol['baseAsset'], [])
    baseSymbols.append(symbol)
    symbolsByBaseDict[symbol['baseAsset']] = baseSymbols
del symbols

In [91]:
from typing import Set
async def initialAssetCandidateSet(year: int, month: int) -> Set[str]:
    symbols = await symbolsTradedEveryDay(year, month)
    candidateAssets = set()
    def getBlacklist() -> List[str]:
        blacklist = None
        with open('blacklist.txt', 'r') as f:
            blacklist = f.read().split(',')
        return blacklist

    blackListSet = set(getBlacklist())
    for symbol in symbols:
        baseAsset = symbolDict[symbol]['baseAsset']
        #if symbolDict[symbol]['status'] == 'TRADING' and (baseAsset.lower() not in blackListSet):
        if symbolDict[symbol]['status'] == 'TRADING':
            candidateAssets.add(baseAsset)
    
    return candidateAssets

In [39]:
t = await initialAssetCandidateSet(2020, 11)
str(t)

"{'WIN', 'STEEM', 'TROY', 'TRB', 'TFUEL', 'BTS', 'IOTA', 'RSR', 'ARPA', 'SOL', 'NXS', 'SRM', 'YFII', 'SUSHI', 'AION', 'XLM', 'SAND', 'VITE', 'ATOM', 'COS', 'VET', 'NMR', 'OXT', 'KAVA', 'GTO', 'KSM', 'SC', 'COMP', 'VIDT', 'CTK', 'AAVE', 'MDT', 'DOGE', 'BTG', 'CRV', 'NKN', 'QSP', 'XRP', 'OCEAN', 'HIVE', 'CHZ', 'XVS', 'BURGER', 'POLY', 'POWR', 'IQ', 'BEAM', 'KEY', 'WNXM', 'DENT', 'CTSI', 'LTC', 'WTC', 'MFT', 'ARDR', 'GAS', 'WABI', 'GO', 'JST', 'BEL', 'WING', 'DOT', 'ANKR', 'CHR', 'ANT', 'IRIS', 'NEBL', 'EGLD', 'HOT', 'LSK', 'DASH', 'SNM', 'UMA', 'NULS', 'BLZ', 'ZIL', 'TCT', 'BAND', 'ALGO', 'PNT', 'ORN', 'CVC', 'QTUM', 'YFI', 'VIB', 'DATA', 'XMR', 'ZEC', 'BNT', 'TOMO', 'NAV', 'COTI', 'KMD', 'FLM', 'VTHO', 'AST', 'HBAR', 'CTXC', 'CREAM', 'INJ', 'GRS', 'DIA', 'AVAX', 'ETH', 'SYS', 'MKR', 'WRX', 'THETA', 'DGB', 'LINK', 'QKC', 'FTM', 'BAT', 'ADA', 'AUD', 'REQ', 'RUNE', 'IDEX', 'ONE', 'SNX', 'SNT', 'PIVX', 'SXP', 'CELR', 'STX', 'PHB', 'WBTC', 'NBS', 'UNI', 'OAX', 'KNC', 'LTO', 'AUDIO', 'SCRT', 

In [92]:
def getTradeSymbols(baseAssetSet):
    symbols = []
    for base in baseAssetSet:
        baseInfo = symbolsByBaseDict[base]
        for el in baseInfo:
            symbols.append(el['symbol'])
    return symbols

In [44]:
getTradeSymbols(t)

['WINBNB',
 'WINBTC',
 'WINUSDT',
 'WINUSDC',
 'WINTRX',
 'WINEUR',
 'WINBRL',
 'WINBUSD',
 'STEEMBTC',
 'STEEMETH',
 'STEEMBNB',
 'STEEMUSDT',
 'STEEMBUSD',
 'TROYBNB',
 'TROYBTC',
 'TROYUSDT',
 'TROYBUSD',
 'TRBBNB',
 'TRBBTC',
 'TRBBUSD',
 'TRBUSDT',
 'TFUELBNB',
 'TFUELBTC',
 'TFUELUSDT',
 'TFUELUSDC',
 'TFUELTUSD',
 'TFUELPAX',
 'TFUELBUSD',
 'BTSBTC',
 'BTSETH',
 'BTSBNB',
 'BTSUSDT',
 'BTSBUSD',
 'IOTABTC',
 'IOTAETH',
 'IOTABNB',
 'IOTAUSDT',
 'IOTABUSD',
 'RSRBNB',
 'RSRBTC',
 'RSRBUSD',
 'RSRUSDT',
 'ARPABNB',
 'ARPABTC',
 'ARPAUSDT',
 'ARPATRY',
 'ARPARUB',
 'ARPABUSD',
 'SOLBNB',
 'SOLBTC',
 'SOLUSDT',
 'SOLBUSD',
 'SOLEUR',
 'SOLGBP',
 'SOLTRY',
 'SOLBRL',
 'SOLRUB',
 'SOLAUD',
 'SOLBIDR',
 'SOLUSDC',
 'SOLETH',
 'NXSBTC',
 'NXSETH',
 'NXSBNB',
 'SRMBNB',
 'SRMBTC',
 'SRMBUSD',
 'SRMUSDT',
 'SRMBIDR',
 'YFIIBNB',
 'YFIIBTC',
 'YFIIBUSD',
 'YFIIUSDT',
 'SUSHIBNB',
 'SUSHIBTC',
 'SUSHIBUSD',
 'SUSHIUSDT',
 'AIONBTC',
 'AIONETH',
 'AIONBNB',
 'AIONBUSD',
 'AIONUSDT',
 'XLMBTC

In [93]:
async def getMedianUSDTTradeVol(baseAssetSet, isoInstantFrom, isoInstantTo):
    def listToStr(l):
        result = l[0]
        for i in range(1, len(l)):
            result += f", {l[i]}"
        return result
    symbolList = getTradeSymbols(baseAssetSet)
    columns = ['symb', 'open_time', 'close', 'volume', 'close_time', 'quote_asset_vol']
    # sql_query = "SELECT %s FROM 'binance_daily' WHERE open_time >= '%s' and open_time < '%s' AND symb IN (%s)"%(listToStr(columns), isoInstantFrom, isoInstantTo, str(symbolList)[1:len(str(symbolList))-1])
    sql_query = "SELECT %s FROM 'binance_daily' WHERE open_time >= '%s' and open_time < '%s'"%(listToStr(columns), isoInstantFrom, isoInstantTo)    
    #print(sql_query)
    query_params = {'query': sql_query, 'fmt': 'json'}
    async with httpx.AsyncClient() as client:
            r = await client.get(host + '/exec', params=query_params)
    jsonR = r.json()
    return pd.DataFrame(columns=columns, data=jsonR['dataset'])

In [123]:
df = await getMedianUSDTTradeVol(t, '2020-11-01T00:00:00.000000Z', '2020-12-01T00:00:00.000000Z')
df

,symb,open_time,close,volume,close_time,quote_asset_vol
0,FTTBNB,2020-11-01T00:00:00.000000Z,1.277000e-01,1.289160e+03,2020-11-01T23:59:59.999999Z,1.668520e+02
1,WBTCBTC,2020-11-01T00:00:00.000000Z,9.999800e-01,5.322180e+01,2020-11-01T23:59:59.999999Z,5.320581e+01
2,WINUSDC,2020-11-01T00:00:00.000000Z,9.630000e-05,3.859144e+08,2020-11-01T23:59:59.999999Z,3.568064e+04
3,BELBTC,2020-11-01T00:00:00.000000Z,5.778000e-05,9.035000e+04,2020-11-01T23:59:59.999999Z,5.203148e+00
4,TRBUSDT,2020-11-01T00:00:00.000000Z,2.083800e+01,1.141019e+05,2020-11-01T23:59:59.999999Z,2.320366e+06
...,...,...,...,...,...,...
23592,MKRBNB,2020-11-30T00:00:00.000000Z,1.807400e+01,1.275830e+02,2020-11-30T23:59:59.999999Z,2.321074e+03
23593,ARDRBTC,2020-11-30T00:00:00.000000Z,3.750000e-06,1.574673e+07,2020-11-30T23:59:59.999999Z,6.263127e+01
23594,SCETH,2020-11-30T00:00:00.000000Z,5.520000e-06,2.430436e+07,2020-11-30T23:59:59.999999Z,1.346971e+02
23595,SCBTC,2020-11-30T00:00:00.000000Z,1.800000e-07,1.355248e+08,2020-11-30T23:59:59.999999Z,2.312523e+01


In [124]:
df['baseAsset'] = df.apply(lambda row: symbolDict[row['symb']]['baseAsset'], axis = 1)
df['quoteAsset'] = df.apply(lambda row: symbolDict[row['symb']]['quoteAsset'], axis = 1)
df

,symb,open_time,close,volume,close_time,quote_asset_vol,baseAsset,quoteAsset
0,FTTBNB,2020-11-01T00:00:00.000000Z,1.277000e-01,1.289160e+03,2020-11-01T23:59:59.999999Z,1.668520e+02,FTT,BNB
1,WBTCBTC,2020-11-01T00:00:00.000000Z,9.999800e-01,5.322180e+01,2020-11-01T23:59:59.999999Z,5.320581e+01,WBTC,BTC
2,WINUSDC,2020-11-01T00:00:00.000000Z,9.630000e-05,3.859144e+08,2020-11-01T23:59:59.999999Z,3.568064e+04,WIN,USDC
3,BELBTC,2020-11-01T00:00:00.000000Z,5.778000e-05,9.035000e+04,2020-11-01T23:59:59.999999Z,5.203148e+00,BEL,BTC
4,TRBUSDT,2020-11-01T00:00:00.000000Z,2.083800e+01,1.141019e+05,2020-11-01T23:59:59.999999Z,2.320366e+06,TRB,USDT
...,...,...,...,...,...,...,...,...
23592,MKRBNB,2020-11-30T00:00:00.000000Z,1.807400e+01,1.275830e+02,2020-11-30T23:59:59.999999Z,2.321074e+03,MKR,BNB
23593,ARDRBTC,2020-11-30T00:00:00.000000Z,3.750000e-06,1.574673e+07,2020-11-30T23:59:59.999999Z,6.263127e+01,ARDR,BTC
23594,SCETH,2020-11-30T00:00:00.000000Z,5.520000e-06,2.430436e+07,2020-11-30T23:59:59.999999Z,1.346971e+02,SC,ETH
23595,SCBTC,2020-11-30T00:00:00.000000Z,1.800000e-07,1.355248e+08,2020-11-30T23:59:59.999999Z,2.312523e+01,SC,BTC


In [125]:
from collections import defaultdict
coinGraph = defaultdict(set)
for key in symbolDict:
    symbol = symbolDict[key]
    coinGraph[symbol['baseAsset']].add(symbol['quoteAsset'])
    coinGraph[symbol['quoteAsset']].add(symbol['baseAsset'])

In [126]:
# We will use USDT to compare apples to apples
def shortestPathToUSDT(fromBaseAsset):
    if fromBaseAsset == 'USDT':
        return [fromBaseAsset]
    parent = {}
    explored = set()
    frontier = [fromBaseAsset]
    def solution(baseAsset):
        if parent.get(baseAsset) is None:
            return [baseAsset]
        else:
            result = solution(parent[baseAsset])
            result.append(baseAsset)
            return result
  
    while len(frontier) > 0:
        node = frontier.pop(0)
        explored.add(node)
        for coin in coinGraph[node]:
            if coin not in explored and coin not in frontier:
                parent[coin] = node
                if coin == 'USDT':
                    return solution(coin)
                else:
                    frontier.append(coin)
    assert False, "Should never get here"

In [127]:
shortestPathToUSDT('GAS')

['GAS', 'BTC', 'USDT']

In [128]:
valInUSDT = []
for (_, row) in df.iterrows():
    # print("(Base asset: %s, Quote asset: %s)"%(row['baseAsset'],row['quoteAsset']))
    if row['quoteAsset'] == 'USDT':
        val = row['volume'] * row['close']
    else:
        path = shortestPathToUSDT(row['quoteAsset'])
        multiplier = 1.0
        assert len(path) > 1
        i = 0
        while i + 1 < len(path):
            dfFiltered = df[(df['baseAsset'] == path[i])
                                 & (df['quoteAsset'] == path[i + 1])
                                 & (df['open_time'] == row['open_time'])]
            if dfFiltered.shape[0] > 0:
                # print(dfFiltered)
                pathRow = dfFiltered.iloc[0]
                multiplier *= pathRow['close']
            else:
                dfFiltered = df[(df['baseAsset'] == path[i + 1]) 
                        & (df['quoteAsset'] == path[i])
                        & (df['open_time'] == row['open_time'])]
                # print(dfFiltered)
                pathRow = dfFiltered.iloc[0]
                multiplier *= 1.0 / pathRow['close']
            i += 1
        assert path[i] == 'USDT'
        val = row['volume'] * row['close'] * multiplier
    valInUSDT.append(val)
df.insert(df.shape[1], 'valInUSDT', valInUSDT)
df

,symb,open_time,close,volume,close_time,quote_asset_vol,baseAsset,quoteAsset,valInUSDT
0,FTTBNB,2020-11-01T00:00:00.000000Z,1.277000e-01,1.289160e+03,2020-11-01T23:59:59.999999Z,1.668520e+02,FTT,BNB,4.697068e+03
1,WBTCBTC,2020-11-01T00:00:00.000000Z,9.999800e-01,5.322180e+01,2020-11-01T23:59:59.999999Z,5.320581e+01,WBTC,BTC,7.323972e+05
2,WINUSDC,2020-11-01T00:00:00.000000Z,9.630000e-05,3.859144e+08,2020-11-01T23:59:59.999999Z,3.568064e+04,WIN,USDC,3.714126e+04
3,BELBTC,2020-11-01T00:00:00.000000Z,5.778000e-05,9.035000e+04,2020-11-01T23:59:59.999999Z,5.203148e+00,BEL,BTC,7.184085e+04
4,TRBUSDT,2020-11-01T00:00:00.000000Z,2.083800e+01,1.141019e+05,2020-11-01T23:59:59.999999Z,2.320366e+06,TRB,USDT,2.377656e+06
...,...,...,...,...,...,...,...,...,...
23592,MKRBNB,2020-11-30T00:00:00.000000Z,1.807400e+01,1.275830e+02,2020-11-30T23:59:59.999999Z,2.321074e+03,MKR,BNB,7.262197e+04
23593,ARDRBTC,2020-11-30T00:00:00.000000Z,3.750000e-06,1.574673e+07,2020-11-30T23:59:59.999999Z,6.263127e+01,ARDR,BTC,1.163046e+06
23594,SCETH,2020-11-30T00:00:00.000000Z,5.520000e-06,2.430436e+07,2020-11-30T23:59:59.999999Z,1.346971e+02,SC,ETH,8.273114e+04
23595,SCBTC,2020-11-30T00:00:00.000000Z,1.800000e-07,1.355248e+08,2020-11-30T23:59:59.999999Z,2.312523e+01,SC,BTC,4.804702e+05


In [129]:
df = df[['open_time', 'baseAsset', 'valInUSDT']].groupby(['baseAsset', 'open_time'], as_index=False).sum()
# df = df[['open_time', 'baseAsset', 'valInUSDT']].groupby(['baseAsset', 'open_time'], as_index=False).sum().sort_values(['baseAsset', 'open_time']).set_index(['baseAsset', 'open_time'])

In [130]:
df

,baseAsset,open_time,valInUSDT
0,AAVE,2020-11-01T00:00:00.000000Z,3.883287e+06
1,AAVE,2020-11-02T00:00:00.000000Z,8.351446e+06
2,AAVE,2020-11-03T00:00:00.000000Z,4.287112e+06
3,AAVE,2020-11-04T00:00:00.000000Z,3.590022e+06
4,AAVE,2020-11-05T00:00:00.000000Z,1.069242e+07
...,...,...,...
7715,ZRX,2020-11-26T00:00:00.000000Z,1.297749e+07
7716,ZRX,2020-11-27T00:00:00.000000Z,4.116352e+06
7717,ZRX,2020-11-28T00:00:00.000000Z,4.290210e+06
7718,ZRX,2020-11-29T00:00:00.000000Z,2.343936e+06
